In [1]:
import pandas as pd
import requests
import numpy as np
import hashlib
from datetime import datetime
from xml.etree.ElementTree import parse
import pytz 
import os
#update
from apscheduler.schedulers.blocking import BlockingScheduler
#GenericDirectories
from pathlib import Path

home_directory = str(Path.home())
print(home_directory)
DownloadedFilesDirectory=home_directory+'/FBDirectory/'
ExportedFilesDirectory=home_directory+'/FBFinalDirectory/'
# define the name of the directory to be created
Directories=[DownloadedFilesDirectory,ExportedFilesDirectory]
for d in Directories:
    if not(os.path.exists(d)):
        try:
            os.mkdir(d)
            print("succesfully created "+d)
        except OSError:
            print ("Creation of the directory failed " +d)
    else:
        print ("Directory exists already " +d)
def update():
    def _get_fb_id(row):
        '''
        Private util method to construct fb_id (fusionbase id) by hashing row values.
        '''
        concat_str = ''.join([str(x) for x in np.array(row)])

        return hashlib.sha3_256(concat_str.encode('utf-8')).hexdigest()[:32]

    URL = "https://seekingalpha.com/feed.xml"
    response = requests.get(URL)

    with open(DownloadedFilesDirectory+'feed.xml', 'wb') as file:
        file.write(response.content)
    document = parse(DownloadedFilesDirectory+'feed.xml')

    title=[]
    guid=[]
    link=[]
    categS=[]
    categA=[]
    pubDate=[]
    #collecting data from the Xml file and save it in lists
    for item in document.iterfind('channel/item'):
        title.append(item.findtext('title'))
        guid.append(item.findtext('guid'))
        link.append(item.findtext('link'))
        symb = [x.text for x in item.findall('category/[@type="symbol"]')]
        categS.append(str(symb))
        aut = [x.text for x in item.findall('category/[@type="author"]')]
        categA.append(str(aut))
        pubDate.append(item.findtext('pubDate'))
    #creating dataframe and filling it with lists content    
    df = pd.DataFrame({'title': title, 'guid':guid, 'link':link,'stock_symbol':categS,'stock_author':categA,'publication_datetime':pubDate})
    #transforming data for some columns(stock_symbol,stock_author)
    df['stock_symbol'] = df['stock_symbol'].str.replace(r'\'', '').str.lstrip("[").str.rstrip("]")
    df['stock_author'] = df['stock_author'].str.replace(r'\'', '').str.lstrip("[").str.rstrip("]")
    df['publication_datetime']=pd.to_datetime(df.publication_datetime,utc=True)
    #adding fusionBase Id and datetime
    fbId=[]
    fbId =  np.array([_get_fb_id(row) for row in df.values])
    df.set_index(fbId,inplace=True)
    df.index.set_names('fb_id',inplace=True)
    dateTime=[]
    for i in range(len(df)):
            dateTime.append(datetime.utcnow())
    df['fb_datetime']=dateTime
    dftemp=df
    if not(os.path.exists(ExportedFilesDirectory+'feed.parquet.gzip')):
            #Creating a new parquet file and filling it with data recently extracted
            dftemp.to_parquet(ExportedFilesDirectory+'feed.parquet.gzip',compression='gzip',engine='fastparquet',index=True)
            print('created')

    else:

            #extracting data from the existing parquet file 
            df=pd.read_parquet(ExportedFilesDirectory+'feed.parquet.gzip',engine='fastparquet')
            #Checking for new updates
            if dftemp.iloc[0,5]!=df.iloc[0,5]:
                #Updating parquet file
                df=pd.concat([dftemp,df],axis=0)
                df.drop_duplicates(subset='publication_datetime',keep='last')
                df.to_parquet(ExportedFilesDirectory+'feed.parquet.gzip',compression='gzip',engine='fastparquet',index=True)
                print('updated')
            else:
                #No updates
                print('no updates')
scheduler = BlockingScheduler()
scheduler.add_job(update, 'interval', minutes=1)
scheduler.start()

C:\Users\Adem
succesfully created C:\Users\Adem/FBDirectory/
succesfully created C:\Users\Adem/FBFinalDirectory/
created
no updates
no updates
no updates
no updates
no updates
no updates
no updates
c
updated
no updates
no updates
no updates
no updates
no updates
no updates
no updates
no updates
no updates
c
updated
no updates
no updates
no updates
no updates
no updates
no updates
no updates
no updates
c
updated
no updates
no updates
no updates
no updates
no updates
c
updated
no updates
no updates
no updates
no updates
c
updated
no updates
no updates
no updates
c
updated
no updates
no updates
no updates
no updates
no updates
no updates
no updates
no updates
no updates
no updates
no updates
no updates
no updates
no updates
no updates
c
updated
no updates
no updates
no updates
no updates
c
updated
no updates
no updates
no updates
no updates
c
updated
no updates
no updates
no updates
c
updated
no updates
no updates
no updates
no updates
no updates
no updates


Job "update (trigger: interval[0:01:00], next run at: 2020-12-02 20:48:33 CET)" raised an exception
Traceback (most recent call last):
  File "C:\Users\Adem\Anaconda3\lib\site-packages\urllib3\connection.py", line 159, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "C:\Users\Adem\Anaconda3\lib\site-packages\urllib3\util\connection.py", line 80, in create_connection
    raise err
  File "C:\Users\Adem\Anaconda3\lib\site-packages\urllib3\util\connection.py", line 70, in create_connection
    sock.connect(sa)
OSError: [WinError 10065] Une opération a été tentée sur un hôte impossible à atteindre

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adem\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "C:\Users\Adem\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 343, in _make_request
    self._validate_conn(conn)
  File "

c
updated
no updates
no updates
no updates
c
updated
no updates


KeyboardInterrupt: 